In [55]:
import sys
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
# from nltk.tokenize.punkt import PunktWordTokenizer
import gensim
from gensim.models.word2vec import Word2Vec
import re
from sklearn.manifold import TSNE
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import show, figure
from string import digits

In [56]:
# Punctuation and Tokenizer module
# nltk.download('punkt')
# The Gutenberg dataset. A set of 18 books we can used
# to train upon.
# nltk.download('gutenberg')
# from nltk.corpus import gutenberg

In [57]:
path ='./'
dataset = pd.read_csv(path + 'train.csv'.format(1), engine='python')
dataset.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [58]:
dataset['toxic'][dataset['toxic'] == 1].value_counts()

1    15294
Name: toxic, dtype: int64

In [59]:
from sklearn.utils import resample

# Separate majority and minority classes
dataset_majority = dataset[dataset.toxic == 0]
dataset_minority = dataset[dataset.toxic == 1]
 
# # Upsample minority class
# dataset_minority_upsampled = resample(dataset_minority, 
#                                  replace = True,  # sample with replacement
#                                  n_samples = 144277,   # to match majority class
#                                  random_state = 123) # reproducible results
 
# # Combine majority class with upsampled minority class
# dataset_upsampled = pd.concat([dataset_majority, dataset_minority_upsampled])
 
# # Display new class counts
# dataset_upsampled.toxic.value_counts()

# Downsample majority class
dataset_majority_downsampled = resample(dataset_majority, replace = True, n_samples = len(dataset_minority),
                                        random_state = 123)

# Combine two classes
dataset_ds = pd.concat([dataset_minority, dataset_majority_downsampled])

# Shuffle dataset
dataset_ds = dataset_ds.sample(frac=1).reset_index(drop=True)

In [60]:
dataset_ds

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,9df6e193b6865375,"Jack and Jill \nEric, thank you for your contr...",1,0,1,0,1,0
1,5ab6b0c097395330,Do you have any source for it?,0,0,0,0,0,0
2,c33869e4d04ea208,Bahariasauridae needs its own article \nFor so...,0,0,0,0,0,0
3,b5881a9dc2e15b0f,"Gobble, gobble, gobble, gobble, gobble. -",0,0,0,0,0,0
4,c14a9a082b6272fe,Not as pissed as Ned Scott will be when I stic...,1,0,1,0,1,0
5,77edf9a2a3e2a56f,So what are you saying here? That we should me...,0,0,0,0,0,0
6,8a5686ca7d7eff3b,Clearly ZIONIST-REVISIONIST MOVEMENT is the co...,0,0,0,0,0,0
7,1e369e926cb5cfdf,Kevin Forsyth hates blacks. Forsyth was forced...,1,0,0,0,0,0
8,90f1d7dd20a66323,That blanket prejudice against everyone from t...,0,0,0,0,0,0
9,11b4e04b309f4d79,"""I would like to extend a fat, robust FUCK YOU...",1,1,1,0,1,0


In [61]:
# Due to lack of resources, I'm not working with the full Gutenberg 
# dataset (18 books). If you got a GPU, you can just omit the
# ‘fileids’ parameter and all 18 books will be loaded.
# gberg_sents = gutenberg.sents(fileids=['bible-kjv.txt',
#                               'austen-emma.txt',
#                               'austen-persuasion.txt',
#                               'austen-sense.txt',
#                               'carroll-alice.txt'])

In [62]:
# len(gutenberg.sents(fileids=['bible-kjv.txt',
#                                    'austen-emma.txt',
#                                    'austen-persuasion.txt',
#                                    'austen-sense.txt',
#                                    'carroll-alice.txt']))

In [53]:
# reload(sys)
# sys.setdefaultencoding('ascii')

In [63]:
data_small = dataset_ds[0:100]
# tokenize data, tolowercase, remove stopwords and remove symbols like \n
# print(dataset_ds['comment_text'].iloc[0])
# print(word_tokenize(dataset_ds['comment_text'].iloc[0]))
tokenizer = TreebankWordTokenizer()
# print(tokenizer.tokenize(dataset_ds['comment_text'].iloc[0]))

def low(x):
#     x = x.decode('utf-8')
#     if type(x) != unicode:
#         x = unicode(x)
    # remove numbers
#     filter(lambda x: x.isalpha(), x)
    x = re.sub(r'[^\w]', ' ', x)
    return x.lower()

def tok_and_rem(x):
#     if type(x) != unicode:
#         x = unicode(x)
    s = word_tokenize(x)
    for word in s:
        if word in stopwords.words('english'):
            s.remove(word)
    return s

# to lowercase and remove special characters
# dataset_ds['comment_text'] = dataset_ds['comment_text'].apply(lambda x: low(x))
data_small['comment_text'] = data_small['comment_text'].apply(lambda x: low(x))

# tokenize and remove stopwords
data_small['comment_text'] = data_small['comment_text'].apply(lambda x: tok_and_rem(x))


In [64]:
data_small

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,9df6e193b6865375,"[jack, jill, eric, thank, for, contribution, t...",1,0,1,0,1,0
1,5ab6b0c097395330,"[you, any, source, it]",0,0,0,0,0,0
2,c33869e4d04ea208,"[bahariasauridae, needs, own, article, some, r...",0,0,0,0,0,0
3,b5881a9dc2e15b0f,"[gobble, gobble, gobble, gobble, gobble]",0,0,0,0,0,0
4,c14a9a082b6272fe,"[pissed, as, ned, scott, be, i, stick, pole, h...",1,0,1,0,1,0
5,77edf9a2a3e2a56f,"[what, saying, should, mention, that, illegal,...",0,0,0,0,0,0
6,8a5686ca7d7eff3b,"[clearly, zionist, revisionist, movement, the,...",0,0,0,0,0,0
7,1e369e926cb5cfdf,"[kevin, forsyth, hates, blacks, forsyth, force...",1,0,0,0,0,0
8,90f1d7dd20a66323,"[blanket, prejudice, everyone, south, east, al...",0,0,0,0,0,0
9,11b4e04b309f4d79,"[would, like, extend, fat, robust, fuck, assho...",1,1,1,0,1,0


In [70]:
# size = 64, dimensions
# sg = 1, use Skip-Gram. If zero, it will use CBOW
# window = 10, context words (10 to the left and 10 to the right)
# min_count = 5, ignore words with frequency lower than that
# seed = 42, the answer to the universe, life and everything.
# workers = 2, number of worker threads.
model = Word2Vec(sentences=data_small['comment_text'], size=64, sg=1,
                 window=10, min_count=5, seed=42,
                 workers=2)



In [72]:
# Shows the coordinates of the word ‘house’ in the vector space.
model['problem']
# print(model.most_similar('house'))
# model.similarity('house','place')

array([ 0.19930044,  0.00657094,  0.22152969, -0.28181851, -0.07885263,
        0.11905867,  0.09282296, -0.07265124,  0.18404777, -0.13437259,
        0.0051498 ,  0.0445953 , -0.06126744, -0.23751304,  0.19217405,
        0.14228219, -0.17997339, -0.03410555,  0.09670641,  0.05460636,
        0.20454314, -0.03217662,  0.16521817, -0.10754781,  0.33769104,
       -0.00532076,  0.0514417 , -0.10695076,  0.10625998,  0.03882402,
       -0.07957107,  0.20723054, -0.27451852,  0.1469319 , -0.20131877,
        0.13200974, -0.00586804, -0.2425316 ,  0.00327149, -0.01856712,
        0.0476294 , -0.10539968, -0.04736742, -0.19479495, -0.21539202,
       -0.08536255, -0.04830457,  0.34101579, -0.0414422 , -0.36011857,
        0.12844905,  0.18870409,  0.00998096, -0.11730384, -0.00108148,
       -0.0847674 , -0.21033531, -0.12782523, -0.00348844, -0.03678443,
        0.14693186, -0.10710843, -0.19072817,  0.0381363 ], dtype=float32)